In [ ]:
import glob
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import requests
import csv
import io

In [ ]:
mapCSVURL = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSAWBkVy4eZ0k04ZKEwppDMvCHNlDIxoZXn1H8puT4xqrw-ui4y-rnFRgBaImR7bBg6XGaVul1X279U/pub?gid=1274123795&single=true&output=csv'

def fetchDataCSVFromURL(url):
  response = requests.get(url)
  data = response.content.decode('utf-8')
  dataFile = io.StringIO(data)
  reader = csv.reader(dataFile)
  return [row for row in reader]

data = fetchDataCSVFromURL(mapCSVURL)
print(data)

def get_vietnamese_translation(intent: str) -> str:
  for row in data:
    if row[1] == intent:
      return row[0]

  print('Not found', intent)
  return intent

In [ ]:
def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i, y[i] + 2, y[i], ha = 'center', rotation=90)

In [ ]:

# Define the path to the NLU YAML files
nlu_files_path = "../data/*.yml"

# Initialize an empty dictionary to store the intent examples count
intent_examples_count = {}

print(glob.glob(nlu_files_path))

# Iterate over each NLU YAML file
for file_path in glob.glob(nlu_files_path):
  with open(file_path, 'r') as file:
    nlu_data = yaml.safe_load(file)
    for intent in nlu_data['nlu']:
      try:
        intent_name = intent['intent']
        if (intent_name == 'greet' or intent_name == 'thanks'):
          continue
        examples_count = len(intent['examples'].split('\n'))
        if intent_name in intent_examples_count:
          intent_examples_count[intent_name] += examples_count
        else:
          intent_examples_count[intent_name] = examples_count

      except KeyError:
        pass

In [ ]:
# Convert the intent examples count dictionary to a pandas DataFrame
df = pd.DataFrame.from_dict(intent_examples_count, orient='index', columns=['Example Count'])
df['intent_vietnamese'] = ''

for index, row in df.iterrows():
  intent_name = index
  intent_vietnamese = get_vietnamese_translation(intent_name)  # Replace with your translation function
  df.at[index, 'intent_vietnamese'] = intent_vietnamese

In [ ]:

# Plot the example quantity of each intent
df.plot(x='intent_vietnamese', y='Example Count', kind='bar', figsize=(30, 6))
plt.xlabel('Intent')
plt.yticks(rotation=90)
plt.title(f'Example Quantity of Each Intent (Total examples: {df["Example Count"].sum()})')

addlabels(df.index, df['Example Count'])


plt.show()